In [1]:
import pandas as pd
from pandas_datareader import data
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings

filterwarnings('ignore')

/usr/local/anaconda3/envs/EDA/lib/python3.8/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


## Data scraping and prep

We first download the tickers of the S&P100 index from Wikipedia and use that to read the data of stock prices

In [2]:
try:
    tickers = pd.read_csv('tickers_sp100.csv')
except FileNotFoundError:    
    website_data = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')
    tickers = website_data[2].Symbol # this is the ticker table
    tickers.to_csv('tickers_sp100.csv', index=False)
    
tickers = tickers.Symbol.to_list()

In [3]:
start_date = '2015-05-31'
end_date = '2020-05-31'

try:
    raw_data = pd.read_pickle('sp100_adj_close.pkl')
except FileNotFoundError:
    raw_data = pd.DataFrame()
    
    print('Downloading started:')
    print('-'*50)
    
    for i, ticker in enumerate(tickers):
        try:
            col = data.DataReader(ticker, 'yahoo', start_date, end_date)['Adj Close']
            raw_data[ticker] = col
        except:
            print(f'Could not retrieve the data for {ticker}')
      
    print('Downloading complete!')
    print('-'*50)
    
    raw_data.to_pickle('sp100_adj_close.pkl')
        

In [4]:
raw_data.head()

,AAPL,ABBV,ABT,ACN,ADBE,AIG,ALL,AMGN,AMT,AMZN,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2015-06-01,120.306801,53.367821,44.218430,87.960396,79.870003,52.126572,60.783951,137.054474,85.369728,430.920013,...,110.690834,90.819168,84.684486,37.619061,66.648201,39.303658,76.173103,47.192017,66.173225,68.942108
2015-06-02,119.772255,53.120647,44.218430,88.088417,79.820000,53.014893,60.974697,136.539520,85.297272,430.989990,...,108.742249,90.525146,84.641922,37.759136,66.580704,39.239769,76.483177,47.309910,65.996109,68.950233
2015-06-03,119.919716,53.431610,44.146141,88.682808,80.129997,54.285191,61.510612,137.141739,83.965469,436.589996,...,109.296326,90.872620,85.637917,38.494534,66.493904,39.191860,76.031357,47.924656,66.314903,68.796333
2015-06-04,119.219307,53.758537,43.983513,87.722641,78.750000,53.841042,60.902039,137.979599,85.768349,430.779999,...,107.726387,90.649872,85.212280,38.074310,65.780273,38.409309,75.526375,47.276230,65.659630,68.205154
2015-06-05,118.564957,53.742588,43.775707,87.237999,78.860001,53.743324,60.384300,137.246475,83.820503,426.950012,...,107.615570,91.131012,85.561317,38.512047,65.934570,37.714581,74.321518,47.672012,64.694435,68.253754


Plan of attack:
 - compute returns of the above (maybe log returns?)
 - compute the correlation matrix and the eigenvalue decomp 
 - go through Marcenko Pasteur and see what it says here

In [5]:
raw_data.describe()

,AAPL,ABBV,ABT,ACN,ADBE,AIG,ALL,AMGN,AMT,AMZN,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
count,1259.000000,1259.000000,1259.000000,1259.000000,1259.000000,1259.000000,1259.000000,1259.000000,1259.000000,1259.000000,...,1259.000000,1259.000000,1259.000000,1259.000000,1259.000000,1259.000000,1259.000000,1259.000000,1259.000000,1259.000000
mean,163.263256,67.823829,57.032421,138.737744,189.240564,51.125631,82.266292,167.144224,141.994596,1254.301025,...,194.327947,119.758237,99.565782,45.663922,117.036017,46.970794,66.598840,46.202574,83.115716,68.831676
std,59.313152,16.006028,17.936347,34.977303,88.204709,7.767584,17.504492,28.302846,48.728925,547.699304,...,60.019195,34.744912,9.812296,6.669402,40.865581,6.950928,10.511350,5.739751,20.806080,7.736165
min,84.962914,39.132446,33.393620,84.256889,73.849998,18.780001,51.764465,115.954842,76.944695,423.500000,...,101.996803,62.097809,76.581131,28.590313,64.343323,34.611927,37.920132,22.530001,50.300903,30.851805
25%,109.506489,52.830814,40.837492,108.450508,100.610001,49.684240,63.788776,143.137283,99.606098,759.799988,...,132.833176,87.262360,92.548512,38.789354,77.806049,42.191633,59.481596,43.538275,64.727516,66.734493
50%,154.996933,67.298393,53.199696,138.632599,176.830002,52.386948,86.281830,163.793625,132.968384,1152.349976,...,204.798050,111.491180,98.882912,47.787170,109.931412,44.747089,70.083092,46.943851,81.237228,70.434479
75%,197.468498,82.467556,73.593513,163.113335,264.134995,56.112165,94.886726,184.233635,174.033318,1773.630005,...,245.589333,153.143166,106.890724,50.376106,147.640671,54.042852,74.808178,49.410133,97.317158,73.459705
max,326.316681,108.304718,98.000000,214.950119,386.600006,61.735287,124.262703,241.108017,258.520020,2497.939941,...,304.850006,185.056152,122.933098,59.545845,212.953430,60.777058,85.652672,59.722210,131.750458,79.808060


In [8]:
raw_data.isna()

,AAPL,ABBV,ABT,ACN,ADBE,AIG,ALL,AMGN,AMT,AMZN,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2015-06-01,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2015-06-02,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2015-06-03,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2015-06-04,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2015-06-05,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-22,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2020-05-26,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2020-05-27,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
raw_data.columns[raw_data.isna().any()]

Index(['DOW', 'KHC', 'PYPL'], dtype='object')

In [13]:
raw_data.index[raw_data.isna().any(axis = 1)]

DatetimeIndex(['2015-06-01', '2015-06-02', '2015-06-03', '2015-06-04',
               '2015-06-05', '2015-06-08', '2015-06-09', '2015-06-10',
               '2015-06-11', '2015-06-12',
               ...
               '2019-03-06', '2019-03-07', '2019-03-08', '2019-03-11',
               '2019-03-12', '2019-03-13', '2019-03-14', '2019-03-15',
               '2019-03-18', '2019-03-19'],
              dtype='datetime64[ns]', name='Date', length=957, freq=None)